In [14]:
from tqdm import tqdm
from skimage.transform import resize
from ply_creation_lib import create_ply
from skimage.measure import block_reduce
import matplotlib.pyplot as plt
import pydicom as dicom
import nibabel as nib
import torchio as tio
import numpy as np
import pickle
import time
import copy
import json
import os

In [18]:
def read_nii(file_path: str) -> np.ndarray:
    nii = nib.load(file_path)
    data = nii.get_fdata()
    original_spacing = nii.header.get_zooms()
    print(f'nii spacing: {original_spacing}')

    target_spacing = 0.25

    # Compute new z size
    z_zoom = original_spacing[2] / target_spacing
    new_z = int(round(data.shape[2] * z_zoom))

    # Resize only along z-axis
    resampled_data = resize(
        data,
        output_shape=(data.shape[0], data.shape[1], new_z),
        order=1,
        preserve_range=True,
        anti_aliasing=False
    ).astype(data.dtype)

    resampled_data[resampled_data < 0] = 0

    data = np.rot90(resampled_data, k=1, axes=(0, 2))



    return data

def read_dicom(input_folder: str) -> np.ndarray:
    files: list[str] = os.listdir(input_folder)
    data = [dicom.dcmread(f'{input_folder}\\{file}') for file in files if file.endswith('.dcm')]


    xy_spacing = data[0].PixelSpacing
    z_spacing = data[0].ImagePositionPatient[2] - data[1].ImagePositionPatient[2]
    print(z_spacing)
    print(f'dcm spacing: {list(xy_spacing) + [z_spacing]}')

    image = np.array([dicom.pixel_array(datum) for datum in data])
    image[image < 0] = 0
    return image

dicom_folder = '20240923_84'
nii_file = 'd:\\dicom\\nii_dataset_original_data\\all\\1.img.nii.gz'

# dcm_array = read_dicom(dicom_folder)
# print(dcm_array.shape)

nii_array = read_nii(nii_file)
print(nii_array.shape)

# create_ply(dcm_array, 'a_dcm.ply')
create_ply(nii_array, 'a_nii.ply')

nii spacing: (0.37695312, 0.37695312, 0.5)
(550, 512, 512)


  1%|▋                                                | 891072/60270665 [00:02<02:58, 332914.49it/s]


KeyboardInterrupt: 